# <center> 1차 크롤링 - <원티드> </center>

* 기업이름
* 각 기업들 모집 링크

In [1]:
from tqdm.notebook import tqdm
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
from urllib.request import urlopen
from urllib.request import Request
import urllib

In [2]:
options = webdriver.ChromeOptions()

url = "https://www.wanted.co.kr/jobsfeed"
driver = webdriver.Chrome(service=Service("../../driver/chromedriver"),
                          options=options)

driver.get(url)

In [3]:
driver.get(url)

In [4]:
#네비게이션바에서 검색버튼 클릭
nav_element = driver.find_element(By.CLASS_NAME, "Aside_asideList__3WNyu")
search_element = nav_element.find_element(By.CLASS_NAME, "SvgIcon_SvgIcon__root__8vwon")

search_element.click()

In [5]:
#검색 창에서 자율주행 검색
search_doc = driver.find_element(By.CLASS_NAME, "SearchInput_SearchInput__gySrv")
search_doc.send_keys('자율주행')
search_doc.send_keys(Keys.ENTER)

In [6]:
#포지션 탭으로 이동
wait = WebDriverWait(driver, 10)
position_tab = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[contains(text(), "포지션")]')))
position_tab.click()

In [17]:
#서울, 경기 필터링(원티드 사이트 selenium으로 필터 적용시에 로그인창이 떠 부득이하게 바꿈)
simple_list = []
wait = WebDriverWait(driver, 10)  
simple_name_elements = []  

while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

    try:
        wait.until(lambda driver: len(driver.find_elements(By.CLASS_NAME, 'JobCard_location__2EOr5')) > len(simple_name_elements))
    except TimeoutException:
        break

    new_elements = driver.find_elements(By.CLASS_NAME, 'JobCard_location__2EOr5')
    simple_name_elements.extend(new_elements)

    # StaleElementReferenceException을 처리하기 위해 엘리먼트 수의 변경을 확인
    try:
        wait.until(lambda driver: len(driver.find_elements(By.CLASS_NAME, 'JobCard_location__2EOr5')) > len(simple_name_elements))
    except TimeoutException:
        break

    if len(simple_name_elements) >= 20:
        break

# 기업 이름 추출 및 출력
simple_names = [element.text.strip() for element in simple_name_elements]
simple_names = simple_names[8:]
print(simple_names)
print(len(simple_names))

['서울 · 한국', '서울 · 한국', '서울 · 한국', '경기 · 한국', '경기 · 한국', '서울 · 한국', '경북 · 한국', '경기 · 한국', '서울 · 한국', '경기 · 한국', '서울 · 한국', '서울 · 한국', '경기 · 한국', '경기 · 한국', '경기 · 한국', '경기 · 한국', '서울 · 한국', '경기 · 한국', '서울 · 한국', '서울 · 한국', '서울 · 한국', '경기 · 한국', '경기 · 한국']
23


In [18]:
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import TimeoutException

# 기업이름 요소 초기 로드
company_name_elements = driver.find_elements(By.CLASS_NAME, 'JobCard_companyName__vZMqJ')
num_elements_before_scroll = len(company_name_elements)

while True:
    # 페이지 스크롤
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2) 

    try:
        # 새로운 회사명 요소들이 로드될 때까지 대기 (타임아웃 10초)
        wait.until(lambda driver: len(driver.find_elements(By.CLASS_NAME, 'JobCard_companyName__vZMqJ')) > num_elements_before_scroll)
    except TimeoutException as ex:
        break

    # 현재 로드된 회사명 요소들 가져오기
    company_name_elements = driver.find_elements(By.CLASS_NAME, 'JobCard_companyName__vZMqJ')
    num_elements_before_scroll = len(company_name_elements)

    if num_elements_before_scroll >= 20:  
        break


company_names = []
href_lists = []

href_elements = driver.find_elements(By.CSS_SELECTOR, 'a[data-attribute-id="position__click"]')

for i, element in enumerate(company_name_elements):
    # simple_names 리스트에서 현재 인덱스에 해당하는 값 가져오기
    simple_name = simple_names[min(i, len(simple_names)-1)]

    # 만약 값이 '서울'이나 '경기'로 시작하면 추가
    if simple_name.startswith('서울') or simple_name.startswith('경기'):
        company_names.append(element.text.strip())
        
        # 현재 인덱스에 해당하는 href 값 가져오기

        href_value = href_elements[i].get_attribute('href')
        href_lists.append(href_value)
      

company_names = company_names[8:]
href_lists = href_lists[8:]
print(f'회사 정보 리스트: {company_names}')
print(f'href 리스트: {href_lists}')

print(len(company_names))
print(len(href_lists))

회사 정보 리스트: ['컨트롤웍스', '뷰런테크놀로지', '베스텔라랩', '씨엔비스', '엑스와이지', '폴라리스쓰리디(Polaris3D)', '에스티피씨', '마스오토', '슈어소프트테크', '알지티', '컨트롤웍스', '슈어소프트테크', '에이모(AIMMO)', '클로봇', '긴트', '뷰런테크놀로지', '에스티피씨', '포티투닷(42dot)', '포티투닷(42dot)', '포티투닷(42dot)', '에스티피씨', '에스티피씨']
href 리스트: ['https://www.wanted.co.kr/wd/190898', 'https://www.wanted.co.kr/wd/199355', 'https://www.wanted.co.kr/wd/111438', 'https://www.wanted.co.kr/wd/196842', 'https://www.wanted.co.kr/wd/199578', 'https://www.wanted.co.kr/wd/183467', 'https://www.wanted.co.kr/wd/189189', 'https://www.wanted.co.kr/wd/198840', 'https://www.wanted.co.kr/wd/198898', 'https://www.wanted.co.kr/wd/182743', 'https://www.wanted.co.kr/wd/190899', 'https://www.wanted.co.kr/wd/198922', 'https://www.wanted.co.kr/wd/132158', 'https://www.wanted.co.kr/wd/162533', 'https://www.wanted.co.kr/wd/141290', 'https://www.wanted.co.kr/wd/199350', 'https://www.wanted.co.kr/wd/169773', 'https://www.wanted.co.kr/wd/150909', 'https://www.wanted.co.kr/wd/118630', 'https://www.wanted.co

In [19]:
driver.quit()